<a href="https://colab.research.google.com/github/habibian/hf_endpoint_handlers/blob/main/custom_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install git-lfs to interact with the repository
!sudo apt-get update
!sudo apt-get install git-lfs
# install transformers (not needed since it is installed by default in the container)
! pip install transformers[sklearn,sentencepiece,audio,vision]


In [ ]:
!git lfs install
!git clone https://huggingface.co/philschmid/distilbert-base-uncased-emotion


Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'distilbert-base-uncased-emotion'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 49 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [ ]:
# setup cli with token
!huggingface-cli login
!git config --global credential.helper store



In [ ]:
%cd /content/distilbert-base-uncased-emotion/

/content/distilbert-base-uncased-emotion


In [ ]:
%%writefile pipeline.py
from typing import Dict, List, Any


class PreTrainedPipeline():
    def __init__(self, path=""):
        # Preload all the elements you are going to need at inference.
        # pseudo:
        # self.model= load_model(path)

    def __call__(self, data: Dict[str, Any]) -> List[Dict[str, Any]]:
        """
       data args:
            inputs (:obj: `str` | `PIL.Image` | `np.array`)
            kwargs
      Return:
            A :obj:`list` | `dict`: will be serialized and returned
        """

        # pseudo
        # self.model(input)

In [ ]:
!echo "holidays" >> requirements.txt
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile pipeline.py
from typing import Dict, List, Any
from transformers import pipeline
import holidays

class PreTrainedPipeline():
    def __init__(self, path=""):
        self.pipeline = pipeline("text-classification",model=path)
        self.holidays = holidays.US()

    def __call__(self, data: Dict[str, Any]) -> List[Dict[str, Any]]:
        """
       data args:
            inputs (:obj: `str`)
            date (:obj: `str`)
      Return:
            A :obj:`list` | `dict`: will be serialized and returned
        """
        # get inputs
        inputs = data.pop("inputs",data)
        date = data.pop("date", None)

        # check if date exists and if it is holiday
        if date is not None and date in self.holidays:
          return [{"label": "happy", "score": 1}]


        # run normal prediction
        prediction = self.pipeline(inputs)
        return prediction

Overwriting pipeline.py


In [ ]:
from pipeline import PreTrainedPipeline

# init pipeline
my_pipe = PreTrainedPipeline(path=".")

# prepare sample playoad
non_holiday_payload = {"inputs": "I am quite excited how this will turn out", "date": "2022-08-08"}
holiday_payload = {"inputs": "Today is a though day", "date": "2022-07-04"}

# test the pipeline
non_holiday_pred=my_pipe(non_holiday_payload)
holiday_payload=my_pipe(holiday_payload)

# show results
print("non_holiday_pred", non_holiday_pred)
print("holiday_payload", holiday_payload)

non_holiday_pred [{'label': 'joy', 'score': 0.9985942244529724}]
holiday_payload [{'label': 'happy', 'score': 1}]


In [ ]:
# add all our new files
!git add *
# commit our files
!git commit -m "add custom pipeline"
# push the files to the hub
!git push